In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import bbknn
import os
from scipy import sparse
import matplotlib.pyplot as plt
# from scanpy_base_moudle_update2 import *
# import scrublet as scr
import datetime
import harmonypy as hm

sc.settings.verbosity = 3
#sc.logging.print_versions()
# 设置图片的分辨率以及其他样式
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=12)

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

# Fig1A

In [ ]:
def change_obs_index_v2(x):
    """
    该函数用于修正concatenate样本或数据集后变更的barcodes名。
    """  
    x = x[:-2]
    return x

In [ ]:
adata_epi = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/EPI.h5ad')
adata_epi = sc.AnnData(X=adata_epi.raw.X, var=adata_epi.raw.var, obs = adata_epi.obs)

adata_imm = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/IMM.h5ad')
adata_imm = sc.AnnData(X=adata_imm.raw.X, var=adata_imm.raw.var, obs = adata_imm.obs)

adata_str = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Stromal.h5ad')
adata_str = sc.AnnData(X=adata_str.raw.X, var=adata_str.raw.var, obs = adata_str.obs)

In [ ]:
adata = adata_epi.concatenate([adata_imm,adata_str], batch_categories=['1','2','3'], join="outer")
adata.var = adata.var[[]]
adata

In [ ]:
adata.obs.rename(index=change_obs_index_v2,inplace=True)
adata.obs

In [ ]:
drop_list = ['mean_epi_score', 'mean_T_ILCs_score', 'mean_B_score', 'mean_MNPs_score', 'mean_Mast_score', 'mean_Stromal_score', 'bbknn_batch', 'Compartments', 'leiden', 'leiden-all-0.5', 'leiden-all-1', 'ann', 'count_CD45', 'count_CD3D', 'count_CD3E', 'count_CD79A', 'count_CD79B', 'count_EPCAM', 'leiden-all-1.5']
adata.obs = adata.obs.drop(drop_list, axis=1)
adata

In [ ]:
adata.obs['annotation'] = adata.obs['annotation'].astype('category')
adata.obs['annotation'].cat.categories

In [ ]:
for i in list(adata.obs['annotation'].cat.categories):
    
    single_index = adata.obs.loc[adata.obs['annotation'].isin([i]), :].index
    
    if i.split('-')[0] == 'C01':
        adata.obs.at[single_index,'Compartment']='Epithelium'
    elif i.split('-')[0] == 'C02':
        adata.obs.at[single_index,'Compartment']='T/ILCs'
    elif i.split('-')[0] == 'C03':
        adata.obs.at[single_index,'Compartment']='B cells'
    elif i.split('-')[0] == 'C04':
        adata.obs.at[single_index,'Compartment']='MNPs'
    elif i.split('-')[0] == 'C05':
        adata.obs.at[single_index,'Compartment']='Mast cells'
    elif i.split('-')[0] == 'C06':
        adata.obs.at[single_index,'Compartment']='Stromal cells'

In [ ]:
def hvg_regress_scale(adata):
    # 注意，HVG要求输入的矩阵必须是对数化处理的
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    adata = adata[:, adata.var.highly_variable]
    print(adata)
    
    # sc.pp.regress_out(adata, ['S_score', 'G2M_score'])
    
    sc.pp.scale(adata, zero_center=False)

    return adata

def progress_pca_bbknn_umap_tsne(adata, n_pcs=20, batch_key='bbknn_batch'):
    ## 主成分分析降维
    sc.tl.pca(adata, svd_solver='arpack')
    # 在PCA坐标中绘制散点图
    sc.pl.pca(adata)
    # 可视化每个PC对数据总方差的贡献, 这为我们提供了有关为计算单元的邻域关系应考虑的PC数量的信息sc.tl.tsne()
    sc.pl.pca_variance_ratio(adata, log=True)

    # 使用bbknn算法处理批次效应(代替了neighbors)
    # bbknn要求数据已经过标准化和主成分分析
    adata.obs['bbknn_batch'] = [str(i)+'-'+str(j) for i,j in zip(adata.obs['dataset'], adata.obs['batch_name'])]
    sc.external.pp.bbknn(adata, batch_key=batch_key, n_pcs=n_pcs)
    
    # 计算UMAP
    sc.tl.umap(adata)
    # sc.tl.tsne(adata, n_jobs=6)

    return adata

In [ ]:
adata.raw = adata

adata = hvg_regress_scale(adata)
adata = progress_pca_bbknn_umap_tsne(adata)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Compartment'], add_outline=True, palette="tab20_r", frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['annotation'], add_outline=True, palette="tab20_r", frameon=False)

In [ ]:
sc.pl.umap(adata, color=['tissue'], add_outline=True, palette="tab20_r", frameon=False)

In [ ]:
# 修改B细胞的注释，将Plasma改为IgA和IgG Plasma
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Fig1_All_219716.h5ad')
adata

In [ ]:
adata_B = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/B_annotation.h5ad')
adata_B

In [ ]:
# IgA Plasma cells
B04_index = adata_B.obs.loc[adata_B.obs["annotation"].isin(['C03-B04-IgA Plasma cells']), :].index
# IgG Plasma
B05_index = adata_B.obs.loc[adata_B.obs["annotation"].isin(['C03-B05-IgG Plasma cells']), :].index

In [ ]:
adata.obs['annotation'] = adata.obs['annotation'].astype('string')

adata.obs.at[B04_index,'annotation']='C03-B04-IgA Plasma cells'
adata.obs.at[B05_index,'annotation']='C03-B05-IgG Plasma cells'

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['annotation'], add_outline=True, palette="tab20_r", frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['annotation'], add_outline=True, palette="tab20_r", frameon=False)

In [ ]:
adata.write('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Fig1_All_219716.h5ad')

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Fig1_All_219716.h5ad')
adata

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['Compartment'],  # Make a crosstab
                columns=adata.obs['Health'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata.obs['Compartment'],  # Make a crosstab
                columns=adata.obs['tissue'], margins=True)  # Name the count column
Groups_tab_1

In [ ]:
#              inferior(HC) inferior(CR) middle(CR) polyp(CR)
# Epithelium:   9584           22435       14646    30149
# T/ILCs:       1784           9107        15502    20269
# B cells:      80            2092       847      9172
# MNPs:         88           1218       613      4608
# Mast cells:   11           510       307      3316
# Stromal cells:3902        17551     28408    23517

In [ ]:
## Postnatal and Mouse
plot_df_2 = pd.DataFrame(data=[
                   [ 9584, 22435, 14646, 30149],
                   [ 1784, 9107, 15502, 20269],
                   [ 80, 2092, 847, 9172],
                   [ 88, 1218, 613, 4608],
                   [ 11, 510, 307, 3316],
                   [ 3902, 17551, 28408, 23517],
                   ],
     
             columns = ['inferior(HC)','inferior(CR)','middle(CR)','polyp(CR)'],
                          
             index = ['Epithelium','T/ILCs','B cells','MNPs','Mast cells','Stromal cells']
                        )

In [ ]:
from matplotlib import cm, colors

colors2 = plt.cm.viridis(np.linspace(0, 1, 128))
colors3 = plt.cm.BuGn(np.linspace(0,0,1))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

# Figure test1
import seaborn as sns

sns.set(style="ticks", font='Arial', font_scale=1.5)
ax = sns.heatmap(plot_df_2, linewidths=2.0, cmap=mymap) #gist_yarg  BuGn
ax.figure.set_size_inches(5,5)

plt.show()

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Compartment'], add_outline=True, frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=12)

from matplotlib import cm, colors
import colorcet as cc

mymap = colors.LinearSegmentedColormap.from_list('my_colormap', cc.CET_L20)

colors2 = mymap(np.linspace(0.2, 1, 128)) # 30%-100%
colors3 = plt.cm.Greys_r(np.linspace(0.8,0.9,5))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

sc.pl.umap(adata, color=['OXT','AVP','OXTR','HLA-DQA2'], frameon=False, color_map = mymap)
sc.pl.umap(adata, color=['HLA-DQA1','DEFB1','CXCL9','CXCL10'], frameon=False, color_map = mymap)
sc.pl.umap(adata, color=['CXCL11','EREG','DUOX2','LYPD3'], frameon=False, color_map = mymap)

In [ ]:
print(adata.obs['Compartment'].cat.categories)

In [ ]:
old_colors = np.array(adata.uns['Compartment_colors'])

new_colors = old_colors
# B cells
new_colors[[0]] = '#2276b2'
# Epithelium
new_colors[[1]] = '#f2a51f'
# MNPs/DCs
new_colors[[2]] = '#cc2028'
# Mast cells
new_colors[[3]] = '#d67ab0'
# Stromal cells
new_colors[[4]] = '#89562b'
# T/ILCs
new_colors[[5]] = '#427637'

adata.uns['Compartment_colors'] = new_colors

In [ ]:
sc.settings.set_figure_params(dpi=600, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Compartment'], add_outline=True, frameon=False)

In [ ]:
print(adata.obs['tissue'].cat.categories)

old_colors = np.array(adata.uns['tissue_colors'])
new_colors = old_colors

# inferior turbinate
new_colors[[0]] = '#ff0000'
# middle turbinate
new_colors[[1]] = '#0077b2'
# polyp
new_colors[[2]] = '#60b55c'

adata.uns['tissue_colors'] = new_colors

sc.settings.set_figure_params(dpi=400, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['tissue'], frameon=False, title='')

In [ ]:
sc.pl.umap(adata, color=['tissue'], groups = ['polyp'], frameon=False, title ='')

In [ ]:
sc.pl.umap(adata, color=['Disease_state'], frameon=False, title='')

In [ ]:
sc.pl.umap(adata, color=['Health'], frameon=False, title='')

In [ ]:
adata.uns['Health_colors']

In [ ]:
print(adata.obs['tissue'].cat.categories)

old_colors = np.array(adata.uns['Health_colors'])
new_colors = old_colors

# HC
new_colors[[0]] = '#1f77b4'
# 
new_colors[[1]] = '#60b55c'

adata.uns['Health_colors'] = new_colors

sc.settings.set_figure_params(dpi=400, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Health'], frameon=False, title='')

In [ ]:
sc.pl.umap(adata, color=['Health'], groups = ['healthy control'], frameon=False, title ='')

# Fig1B

In [ ]:
adata = sc.read('/home/wangyue/basic-calculation_data/c_Project_outputs/qilu_CRSwNP/Fig1_All_219716.h5ad')

In [ ]:
adata

In [ ]:
adata.obs['annotation'].cat.categories

In [ ]:
# 从所有细胞中，提取SMG, basal cell, KRT14+EMT cell, Fibroblast.

E_index = adata.obs.loc[adata.obs["annotation"].isin(['C01-E03-Club','C01-E11-LY6D+Club cell',
                                                      'C01-E05-Glandular cell', 'C01-E06-MUC5B+GC','C01-E07-MMP7+GC', 'C01-E08-PRB1+GC',
                                                      'C06-S06-Fibroblast','C06-S10-PIEZO2+fibroblast','C06-S11-OXTR+fibroblast','C06-S12-Activated fibroblast']), :].index
F = adata[E_index, :]
F

In [ ]:
import cosg as cosg
import time
t0= time.clock()
cosg.cosg(F,
    key_added='cosg',
        mu=1,
        n_genes_user=50,
               groupby='annotation')
runtime_cosg = time.clock() - t0

sc.settings.set_figure_params(dpi=200, figsize = (4, 4), fontsize=20)
sc.pl.rank_genes_groups_dotplot(F,groupby='annotation',
                                cmap='Spectral_r',
                                 standard_scale='var',
                                       n_genes=4,key='cosg')

In [ ]:
import cosg as cosg

cosg.cosg(F,
    key_added='cosg',
        mu=1,
        n_genes_user=50,
               groupby='annotation')

In [ ]:
sc.settings.set_figure_params(dpi=200, figsize = (4, 4), fontsize=20)
df_tmp=pd.DataFrame(F.uns['cosg']['names'][:4,]).T
df_tmp.reindex(F.obs['annotation'].cat.categories)
marker_genes_list=np.ravel(df_tmp.reindex(F.obs['annotation'].cat.categories))
sc.pl.dotplot(F, marker_genes_list,
             groupby='annotation',              
             dendrogram=False,
             standard_scale='var',
             cmap='Spectral_r')

In [ ]:
categories_order = ['C01-E01-Basal resting','C01-E11-S100A8+APC','C01-E12-KRT14+EMT cell','C01-E05-SMG','C01-E06-MUC5B+SMG','C01-E07-MMP7+SMG', 'C01-E08-PRB1+SMG','C06-S06-Fibroblast','C06-S10-PIEZO2+fibroblast','C06-S11-OXTR+fibroblast']

In [ ]:
# SMG在NP中上调CCL28，趋化嗜酸性粒细胞（CCR3）和Plasma cell
marker_genes_list = ['KRT17','KRT15',
                     'LY6D','S100A8','IL1RN','CXCL9','HLA-DQA2','EREG',
                     'KRT14','MYLK','MYH11','COL9A3',
                     'LYZ','AZGP1',
                     'MUC5B','BPIFB2','TFF1',
                     'MMP7','LCN2','CCL28',
                     'PRB1','PRB2','PRB3',
                     'DCN','FBLN1',
                     'PIEZO2','TTR','SLC18A2',
                     'OXTR','ALOX5AP','COL10A1']

In [ ]:
sc.settings.set_figure_params(dpi=300,fontsize=20)
mp = sc.pl.dotplot(F, 
              marker_genes_list, 
              'annotation', 
              dendrogram=False, 
              #figsize=(3, 4),
              use_raw=True,
              cmap = 'Reds',
              var_group_rotation=45,
              #swap_axes=True,
              categories_order=categories_order,
              standard_scale='var',
              colorbar_title=None,
              return_fig=True
              # , save='_' + sample_name + '_fig11.png'
             ) 
mp.style(grid=True,cmap = 'Spectral_r').show() # RdYlBu_r

In [ ]:
categories_order = ['C01-E03-Club','C01-E11-LY6D+Club cell','C01-E05-Glandular cell', 'C01-E06-MUC5B+GC','C01-E07-MMP7+GC', 'C01-E08-PRB1+GC','C06-S06-Fibroblast','C06-S10-PIEZO2+fibroblast','C06-S11-OXTR+fibroblast','C06-S12-Activated fibroblast']

In [ ]:
# SMG在NP中上调CCL28，趋化嗜酸性粒细胞（CCR3）和Plasma cell
marker_genes_list = ['KRT5','SERPINB3',
                     'LY6D','S100A8','IL1RN','HLA-DQA2','EREG',
                     'LYZ','AZGP1',
                     'MUC5B','BPIFB2','TFF1',
                     'MMP7','LCN2','CCL28',
                     'PRB1','PRB2','PRB3',
                     'DCN','FBLN1',
                     'PIEZO2','TTR','SLC18A2',
                     'OXTR','ALOX5AP','COL10A1',
                     'IDO1','MMP3','CXCL10']

In [ ]:
import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
sc.settings.set_figure_params(dpi=300,fontsize=20)
mp = sc.pl.dotplot(F, 
              marker_genes_list, 
              'annotation', 
              dendrogram=False, 
              #figsize=(3, 4),
              use_raw=True,
              cmap = 'Reds',
              var_group_rotation=45,
              #swap_axes=True,
              categories_order=categories_order,
              standard_scale='var',
              colorbar_title=None,
              return_fig=True
              # , save='_' + sample_name + '_fig11.png'
             ) 
mp.style(grid=True,cmap = 'Spectral_r').show() # RdYlBu_r

# Fig1C

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Fig1_All_219716.h5ad')
adata

In [ ]:
# 下鼻甲，比较健康和慢性鼻窦炎
E_index = adata.obs.loc[adata.obs["tissue"].isin(['inferior turbinate']), :].index
adata_IT = adata[E_index, :]
adata_IT

In [ ]:
Compartment_list = []

for i in adata_IT.obs['Compartment'].to_list():
    
    if i == 'Epithelium':
        j = '1-Epithelium'
    elif i == 'T/ILCs':
        j = '2-T/ILCs'
    elif i == 'B cells':
        j = '3-B cells'
    elif i == 'MNPs':
        j = '4-MNPs'
    elif i == 'Mast cells':
        j = '5-Mast cells'
    else:
        j = '6-Stromal cells'
        
    Compartment_list.append(j)

adata_IT.obs['Compartment_name'] = Compartment_list

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata_IT.obs['Health'],  # Make a crosstab
                        columns=adata_IT.obs['Compartment_name'], margins=True)               # Name the count column
MyTab_1= Groups_tab_1.div(Groups_tab_1["All"], axis=0)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")
MyTab2_1.T

In [ ]:
old_colors = np.array(adata_IT.uns['Compartment_colors'])

new_colors = old_colors
# B cells
new_colors[[0]] = '#2276b2'
# Epithelium
new_colors[[1]] = '#f2a51f'
# MNPs/DCs
new_colors[[2]] = '#cc2028'
# Mast cells
new_colors[[3]] = '#d67ab0'
# Stromal cells
new_colors[[4]] = '#89562b'
# T/ILCs
new_colors[[5]] = '#427637'

adata_IT.uns['Compartment_colors'] = new_colors

In [ ]:
sc.settings.set_figure_params(dpi=400, figsize = (4, 4), fontsize=7)
MyTab2_1 = MyTab2_1.T
order = ['healthy control', 'polyp']
MyTab2_1 = MyTab2_1[order]
MyTab2_1 = MyTab2_1.T
#categories = IMM_group[::-1]

MyTab2_1.columns = pd.CategoricalIndex(MyTab2_1.columns.values)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_1 = MyTab2_1.sort_index(axis=1)

ax = MyTab2_1.plot.bar(
            figsize=(2.6,5),
            stacked=True,
            edgecolor = '#000000',
            linewidth=0.4,
            width=0.8, 
            fontsize=25,
            color={"1-Epithelium": "#f2a51f",
                   "2-T/ILCs": "#427637","3-B cells": "#2276b2", "4-MNPs": "#cc2028", "5-Mast cells": "#d67ab0",
                   "6-Stromal cells": "#89562b"}
             )


plt.title("", fontsize=12)
plt.ylabel("", fontsize=12)
plt.xlabel("", fontsize=12)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
#plt.xticks([])
#plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=8)
plt.grid(False)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

In [ ]:
# 发炎中鼻甲与鼻息肉对比
# 下鼻甲，比较健康和慢性鼻窦炎
E_index = adata.obs.loc[adata.obs["tissue"].isin(['middle turbinate','polyp']), :].index
adata_MT = adata[E_index, :]
adata_MT

In [ ]:
Compartment_list = []

for i in adata_MT.obs['Compartment'].to_list():
    
    if i == 'Epithelium':
        j = '1-Epithelium'
    elif i == 'T/ILCs':
        j = '2-T/ILCs'
    elif i == 'B cells':
        j = '3-B cells'
    elif i == 'MNPs':
        j = '4-MNPs'
    elif i == 'Mast cells':
        j = '5-Mast cells'
    else:
        j = '6-Stromal cells'
        
    Compartment_list.append(j)

adata_MT.obs['Compartment_name'] = Compartment_list

In [ ]:
Groups_tab_1 = pd.crosstab(index=adata_MT.obs['tissue'],  # Make a crosstab
                        columns=adata_MT.obs['Compartment_name'], margins=True)               # Name the count column
MyTab_1= Groups_tab_1.div(Groups_tab_1["All"], axis=0)
MyTab2_1 = MyTab_1.drop(columns="All")
MyTab2_1 = MyTab2_1.drop(index="All")
MyTab2_1.T

In [ ]:
old_colors = np.array(adata_MT.uns['Compartment_colors'])

new_colors = old_colors
# B cells
new_colors[[0]] = '#2276b2'
# Epithelium
new_colors[[1]] = '#f2a51f'
# MNPs/DCs
new_colors[[2]] = '#cc2028'
# Mast cells
new_colors[[3]] = '#d67ab0'
# Stromal cells
new_colors[[4]] = '#89562b'
# T/ILCs
new_colors[[5]] = '#427637'

adata_MT.uns['Compartment_colors'] = new_colors

In [ ]:
MyTab2_1 = MyTab2_1.T
order = ['middle turbinate', 'polyp']
MyTab2_1 = MyTab2_1[order]
MyTab2_1 = MyTab2_1.T
#categories = IMM_group[::-1]

MyTab2_1.columns = pd.CategoricalIndex(MyTab2_1.columns.values)

# Sort the columns (axis=1) by the new categorical ordering
MyTab2_1 = MyTab2_1.sort_index(axis=1)

ax = MyTab2_1.plot.bar(
            figsize=(2.6,5),
            stacked=True,
            edgecolor = '#000000',
            linewidth=0.4,
            width=0.8, 
            fontsize=25,
            color={"1-Epithelium": "#f2a51f",
                   "2-T/ILCs": "#427637","3-B cells": "#2276b2", "4-MNPs": "#cc2028", "5-Mast cells": "#d67ab0",
                   "6-Stromal cells": "#89562b"}
             )


plt.title("", fontsize=12)
plt.ylabel("", fontsize=12)
plt.xlabel("", fontsize=12)
plt.ylim=1.0

#plt.gca().get_legend().remove() #remove legend
# plt.legend(categories, loc='center left', bbox_to_anchor=(1, 0.6), fontsize=12)
# plt.savefig('Proportion of clusters accross organs.png')
# 去除刻度
#plt.xticks([])
#plt.yticks([])
# ax.tick_params(bottom=False, top=False, left=False, right=False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='center left', bbox_to_anchor=(1, 0.6), fontsize=8)
plt.grid(False)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.show()

# Fig1D

**Regulon Visualization**

In [ ]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt
import loompy as lp

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from IPython.display import HTML, display

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

## TFs

In [ ]:
AUCELL_MTX_FNAME = "/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/auc.csv"
BIN_MTX_FNAME = "/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/bin.csv"
THR_FNAME = "/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/thresholds.csv"
adjacencies_FNAME = '/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/adj.tsv'
MOTIFS_FNAME = '/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/reg.csv'

In [ ]:
auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)
bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col=0)
thresholds = pd.read_csv(THR_FNAME, index_col=0).threshold
adjacencies = pd.read_csv(adjacencies_FNAME, index_col=False, sep='\t')
df_motifs = load_motifs(MOTIFS_FNAME)

In [ ]:
# 最关键的矩阵数据，每个细胞的regulon评分
auc_mtx.head(5)

In [ ]:
adjacencies

In [ ]:
adjacencies[adjacencies['target'] == "IL1RN"].head(30)

In [ ]:
adjacencies[adjacencies['target'] == "OXTR"].head(30)

In [ ]:
search_se = adjacencies.loc[adjacencies['TF'] == "AEBP1", 'target'] == "OXTR"
# 得到筛选后的行的index
search_index = search_se[search_se == True].index
# 得到筛选后的 dataframe
target_dataframe = adjacencies.loc[search_index]
target_dataframe

### TFs specificity scores (RSS) panel plot with defined cell types

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/Fig1_All_219716.h5ad')

tf_GENES = '/mnt/data/project/scenic/auxilliaries/lambert2018_c.txt'
amps_pd = pd.read_table(tf_GENES)
print(len(amps_pd))

amps_list = list(amps_pd['Gene_name'])
amps_list = [x for x in amps_list if x in adata.raw.var_names]
len(amps_list)
adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)
adata = adata[:,amps_list]
adata

In [ ]:
sc.pp.filter_genes(adata, min_cells=5)
adata

In [ ]:
# Z-score
sc.pp.scale(adata, max_value=1)
auc_mtx = adata.to_df()

In [ ]:
# 需要保证后续加上一个数值后所有的数均大于0
np.min(auc_mtx.min().to_list())

In [ ]:
auc_mtx = auc_mtx.applymap(lambda x: x +2)

In [ ]:
# Calculate RSS
rss = regulon_specificity_scores(auc_mtx, adata.obs.annotation)
rss.head()

In [ ]:
rss.T.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/rss.csv')

In [ ]:
len(adata.obs['annotation'].cat.categories)

In [ ]:
from adjustText import adjust_text

cats = list(rss.index)

fig = plt.figure(figsize=(20, 70)) # 长/宽
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss.T[c]
    ax = fig.add_subplot(14,4,num) # 设置3行4列的图
    plot_rss(rss, c, top_n=20, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    plt.grid(False)
    for t in ax.texts:
        t.set_fontsize(12)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
# plt.savefig("PBMC10k_cellType-RSS-top5.pdf", dpi=600, bbox_inches = "tight")

plt.show()

## Regulons

In [ ]:
auc_mtx = pd.read_csv(AUCELL_MTX_FNAME, index_col=0)
bin_mtx = pd.read_csv(BIN_MTX_FNAME, index_col=0)
thresholds = pd.read_csv(THR_FNAME, index_col=0).threshold
adjacencies = pd.read_csv(adjacencies_FNAME, index_col=False, sep='\t')
df_motifs = load_motifs(MOTIFS_FNAME)

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/NP_scenic_19522.h5ad')
adata

In [ ]:
# Calculate RSS
rss = regulon_specificity_scores(auc_mtx, adata.obs.annotation)
rss.head()

In [ ]:
from adjustText import adjust_text

cats = list(rss.index)

fig = plt.figure(figsize=(20, 70)) # 长/宽
for c,num in zip(cats, range(1,len(cats)+1)):
    x=rss.T[c]
    ax = fig.add_subplot(14,4,num) # 设置3行4列的图
    plot_rss(rss, c, top_n=20, max_n=None, ax=ax)
    ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
    plt.grid(False)
    for t in ax.texts:
        t.set_fontsize(12)
    ax.set_ylabel('')
    ax.set_xlabel('')
    adjust_text(ax.texts, autoalign='xy', ha='right', va='bottom', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )
 
fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
plt.tight_layout()
plt.rcParams.update({
    'figure.autolayout': True,
        'figure.titlesize': 'large' ,
        'axes.labelsize': 'medium',
        'axes.titlesize':'large',
        'xtick.labelsize':'medium',
        'ytick.labelsize':'medium'
        })
# plt.savefig("PBMC10k_cellType-RSS-top5.pdf", dpi=600, bbox_inches = "tight")

plt.show()

**Explore downstream target genes network of transcription factors, and the transcription factors upstream of target genes**

In [ ]:
adjacencies = pd.read_csv(adjacencies_FNAME, index_col=False, sep='\t')
adjacencies

In [ ]:
from pyscenic.utils import modules_from_adjacencies
modules = list(modules_from_adjacencies(adjacencies, adata.to_df()))

In [ ]:
df_motifs.head(5)

**获取感兴趣基因的上游调控转录因子**

In [ ]:
regulons = df_motifs
TF_list = []

for i in list(regulons.index):
    TF = i[0]
    TF_list.append(TF)
    
TF_list = list(set(TF_list))
len(TF_list)  # len(TF_list)与auc_mtx的转录因子数目一致

In [ ]:
TARGET_GENE = 'EREG'
TARGET_reg_tf_list = []

for i in TF_list:
    TF = i
    for j in regulons.loc[(TF,slice(None)),:][('Enrichment','TargetGenes')]:
        for k in j:
            if k[0]==TARGET_GENE:
                TARGET_reg_tf_list.append(TF)
                break
            else:
                pass
            
list(set(TARGET_reg_tf_list))

In [ ]:
TARGET_GENE = 'S100A8'
TARGET_reg_tf_list = []

for i in TF_list:
    TF = i
    for j in regulons.loc[(TF,slice(None)),:][('Enrichment','TargetGenes')]:
        for k in j:
            if k[0]==TARGET_GENE:
                TARGET_reg_tf_list.append(TF)
                break
            else:
                pass
            
list(set(TARGET_reg_tf_list))

In [ ]:
TARGET_GENE = 'LY6D'
TARGET_reg_tf_list = []

for i in TF_list:
    TF = i
    for j in regulons.loc[(TF,slice(None)),:][('Enrichment','TargetGenes')]:
        for k in j:
            if k[0]==TARGET_GENE:
                TARGET_reg_tf_list.append(TF)
                break
            else:
                pass
            
list(set(TARGET_reg_tf_list))

In [ ]:
TARGET_GENE = 'IL1RN'
TARGET_reg_tf_list = []

for i in TF_list:
    TF = i
    for j in regulons.loc[(TF,slice(None)),:][('Enrichment','TargetGenes')]:
        for k in j:
            if k[0]==TARGET_GENE:
                TARGET_reg_tf_list.append(TF)
                break
            else:
                pass
            
list(set(TARGET_reg_tf_list))

**层次聚类树**

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import bbknn
import os
from scipy import sparse
import matplotlib.pyplot as plt
# from scanpy_base_moudle_update2 import *
# import scrublet as scr
import datetime
import harmonypy as hm

sc.settings.verbosity = 3
#sc.logging.print_versions()
# 设置图片的分辨率以及其他样式
sc.settings.set_figure_params(dpi=150, figsize = (4, 3), fontsize=12)

import matplotlib.font_manager
flist = matplotlib.font_manager.get_fontconfig_fonts()
names = [matplotlib.font_manager.FontProperties(fname=fname).get_name() for fname in flist]
print(names)

params={
        #'font.style':'italic',
        'font.weight':'normal',    #or 'blod'
        }
plt.rcParams.update(params)

plt.rcParams['font.family']='Arial'

In [ ]:
adata = sc.read('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/NP_scenic_19522.h5ad')
AUCELL_MTX_FNAME = "/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/auc.csv"
aucell_adata = sc.read_csv(AUCELL_MTX_FNAME)
aucell_adata.obs = adata.obs
aucell_adata.raw = aucell_adata
#sc.pp.normalize_total(aucell_adata, target_sum=1e4)
#sc.pp.log1p(aucell_adata)

sc.pp.scale(aucell_adata, zero_center=False)
## 主成分分析降维

sc.tl.pca(aucell_adata, svd_solver='arpack')
aucell_adata

In [ ]:
sc.tl.dendrogram(aucell_adata, 'annotation')

sc.settings.set_figure_params(dpi=600, fontsize=3.5)

# N-1组对比，N为细胞类型数目
sc.pl.dendrogram(aucell_adata, 'annotation')

In [ ]:
z_var = aucell_adata.uns['dendrogram_annotation']['linkage']
categories = aucell_adata.obs['annotation'].cat.categories

import scipy
T = scipy.cluster.hierarchy.to_tree(z_var , rd=False )

# Create dictionary for labeling nodes by their IDs
labels=list(categories)
id2name = dict(enumerate(labels))

In [ ]:
# Load required modules
import pandas as pd 
import scipy.spatial
import scipy.cluster
import numpy as np
import json
import matplotlib.pyplot as plt
from functools import reduce

# Create a nested dictionary from the ClusterNode's returned by SciPy
def add_node(node, parent ):
    # First create the new node and append it to its parent's children
    newNode = dict( node_id=node.id, children=[] )
    parent["children"].append( newNode )

    # Recursively add the current node's children
    if node.left: add_node( node.left, newNode )
    if node.right: add_node( node.right, newNode )

# Initialize nested dictionary for d3, then recursively iterate through tree
d3Dendro = dict(children=[], name="Root1")
add_node( T, d3Dendro )


# Label each node with the names of each leaf in its subtree
def label_tree( n ):
    # If the node is a leaf, then we have its name
    if len(n["children"]) == 0:
        leafNames = [ id2name[n["node_id"]] ]
    
    # If not, flatten all the leaves in the node's subtree
    else:
        leafNames = reduce(lambda ls, c: ls + label_tree(c), n["children"], [])

    # Delete the node id since we don't need it anymore and
    # it makes for cleaner JSON
    del n["node_id"]

    # Labeling convention: "&"-separated leaf names
    n["name"] = name = "&".join(sorted(map(str, leafNames)))
    
    return leafNames

label_tree( d3Dendro["children"][0] )

In [ ]:
tree=[]  # tree用于循环差异基因(转录因子)分析
def preorder(root):
    if not root.is_leaf():
        str1=",".join([list(categories)[i] for i in root.left.pre_order()])
        str2=",".join([list(categories)[i] for i in root.right.pre_order()])
        tree.append([str1,str2])
    if root.get_left() is not None:
        preorder(root.get_left())
    if root.get_right() is not None:
        preorder(root.get_right())
preorder(T)
print(tree)

In [ ]:
group_0_name_list = []
group_1_name_list = []
rank_reg_group0_list = []
rank_reg_group1_list = []

for i in tree:
    group0 = i[0].split(',')
    group_0_name_list.append(group0)
    
    group1 = i[1].split(',')
    group_1_name_list.append(group1)
    
    # group = group0 + group1
    
    group0_index = aucell_adata.obs.loc[aucell_adata.obs["annotation"].isin(group0), :].index
    group1_index = aucell_adata.obs.loc[aucell_adata.obs["annotation"].isin(group1), :].index
    
    # 建立新的dataframe列['group_DEG']，并将group0的index对应的列的值修改为group0
    aucell_adata.obs['group_DEG'] = aucell_adata.obs['annotation']
    # 解决categories报错的问题，将原本不存在的类别添加，并删除不再使用的类别
    new_categories = ['group0']
    aucell_adata.obs['group_DEG'].cat.add_categories(new_categories, inplace = True)
    aucell_adata.obs.loc[group0_index,'group_DEG']='group0'
    aucell_adata.obs['group_DEG'].cat.remove_unused_categories(inplace = True)
    
    # 在dataframe列['group_DEG']修改了group0的基础上，将group1的index对应的列的值修改为group1
    new_categories = ['group1']
    aucell_adata.obs['group_DEG'].cat.add_categories(new_categories, inplace = True)
    aucell_adata.obs.loc[group1_index,'group_DEG']='group1'
    aucell_adata.obs['group_DEG'].cat.remove_unused_categories(inplace = True)

    sc.tl.rank_genes_groups(aucell_adata, 
                        'group_DEG', 
                        method='wilcoxon',
                        groups=['group0'],
                        reference='group1')
    result_DEG = sc.get.rank_genes_groups_df(aucell_adata, 
                                             group="group0")
    
    rank_reg_top10_group0 = list(result_DEG['names'])[0:10]
    rank_reg_group0_list.append(rank_reg_top10_group0)
    
    rank_reg_top10_group1 = list(result_DEG['names'])[-10:][::-1] # group1的top10为最后10个reg的倒序.
    rank_reg_group1_list.append(rank_reg_top10_group1)
    
result_reg = pd.DataFrame()
result_reg['group_0_name_list'] = group_0_name_list
result_reg['group_1_name_list'] = group_1_name_list
result_reg['rank_reg_top10_group0'] = rank_reg_group0_list
result_reg['rank_reg_top10_group1'] = rank_reg_group1_list
result_reg

In [ ]:
result_reg.to_csv('/mnt/data2/Datasets/Human_non_intestine_datasets/Qilu_Otorhinolaryngology_surgery_data/dataset_output/SCENIC/result_reg_All.csv')

**UMAP**

In [ ]:
sc.external.pp.bbknn(aucell_adata, batch_key='batch_name', n_pcs=20, neighbors_within_batch = 1)
sc.tl.umap(aucell_adata)

In [ ]:
aucell_adata

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=7)
sc.pl.umap(aucell_adata, color=['Compartment'] ,add_outline=True, frameon=False)
sc.pl.umap(aucell_adata, color=['annotation'] ,add_outline=True, frameon=False, palette="tab20_r")

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=7)
sc.pl.umap(aucell_adata, color=['Compartment'] ,add_outline=True, frameon=False)
sc.pl.umap(aucell_adata, color=['annotation'] ,add_outline=True, frameon=False, palette="tab20_r")

In [ ]:
adata = sc.read('/home/wangyue/basic-calculation_data/c_Project_outputs/qilu_CRSwNP/Fig1_All_219716.h5ad')
adata = sc.AnnData(X=adata.raw.X, var=adata.raw.var, obs = adata.obs)

tf_GENES = '/home/wangyue/basic-calculation_data/d_Packages/scenic/auxilliaries/lambert2018_c.txt'
amps_pd = pd.read_table(tf_GENES)
print(len(amps_pd))

amps_list = list(amps_pd['Gene_name'])
amps_list = [x for x in amps_list if x in adata.var_names]
len(amps_list)

adata = adata[:,amps_list]
adata

In [ ]:
sc.pp.scale(adata, zero_center=False)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.external.pp.bbknn(adata, batch_key='batch_name', n_pcs=20, neighbors_within_batch = 1)
sc.tl.umap(adata)

In [ ]:
sc.settings.set_figure_params(dpi=400, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Compartment'] ,add_outline=True, frameon=False)
sc.pl.umap(adata, color=['annotation'] ,add_outline=True, frameon=False, palette="tab20_r")

In [ ]:
# 按细胞区室划分颜色
adata.obs['cell_compartment'] = adata.obs['annotation']
sc.pl.umap(adata, color=['cell_compartment'] ,add_outline=True, frameon=False)
new_colors = np.array(adata.uns['cell_compartment_colors'])

# 上皮细胞
new_colors[[0,1,2,3,4,5,6,7,8,9,10,11]] = '#050dfc'
# T-ILC
new_colors[[12,13,14,15,16,17,18,19,20,21,22,23]] = '#fc2819'
# Follicular B
new_colors[[24,25]] = '#02ad05'
# Plasma B
new_colors[[26,27,28]] = '#fccb05'
# MNPs
new_colors[[29,30,31,32,33,34,35,36,37]] = '#fc7c05'
# Mast
new_colors[[38]] = '#393b79'
# Gila
new_colors[[39]] = '#427a4d'
# Endo
new_colors[[40,41,42,43]] = '#05c2fc' # ACKR1+Endo
# Fibro
new_colors[[44,45,46,47,48,49,50,51,52,53]] = '#b02398' # RSPO3+OGN+


adata.uns['cell_compartment_colors'] = new_colors

sc.settings.set_figure_params(dpi=500, figsize = (4, 4), fontsize=8)
sc.pl.umap(adata, color=['cell_compartment'], add_outline=True, frameon=False)

In [ ]:
sc.settings.set_figure_params(dpi=150, figsize = (4, 4), fontsize=7)
sc.pl.umap(adata, color=['Compartment'] ,add_outline=True, frameon=False)
sc.pl.umap(adata, color=['annotation'] ,add_outline=True, frameon=False, palette="tab20_r")

In [ ]:
# 按细胞区室划分颜色
aucell_adata.obs['cell_compartment'] = aucell_adata.obs['annotation']
sc.pl.umap(aucell_adata, color=['cell_compartment'] ,add_outline=True, frameon=False)
new_colors = np.array(aucell_adata.uns['cell_compartment_colors'])

# 上皮细胞
new_colors[[0,1,2,3,4,5,6,7,8,9,10,11]] = '#050dfc'
# T-ILC
new_colors[[12,13,14,15,16,17,18,19,20,21,22,23]] = '#fc2819'
# Follicular B
new_colors[[24,25]] = '#02ad05'
# Plasma B
new_colors[[26,27,28]] = '#fccb05'
# MNPs
new_colors[[29,30,31,32,33,34,35,36,37]] = '#fc7c05'
# Mast
new_colors[[38]] = '#393b79'
# Gila
new_colors[[39]] = '#427a4d'
# Endo
new_colors[[40,41,42,43]] = '#05c2fc' # ACKR1+Endo
# Fibro
new_colors[[44,45,46,47,48,49,50,51,52,53]] = '#b02398' # RSPO3+OGN+


aucell_adata.uns['cell_compartment_colors'] = new_colors

sc.settings.set_figure_params(dpi=500, figsize = (4, 4), fontsize=8)
sc.pl.umap(aucell_adata, color=['cell_compartment'], add_outline=True, frameon=False)

# 20230816-FigS

In [ ]:
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=12)
# Mast
colors2 = plt.cm.Oranges(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['IL1B'], frameon=False, color_map=mymap)

In [ ]:
sc.pl.umap(adata, color=['NR1H4','NR1H3','VDR','GPBAR1','IL1R1'], frameon=False, color_map=mymap)

In [ ]:
adata = sc.read('/home/wangyue/basic-calculation_data/c_Project_outputs/qilu_CRSwNP/Fig1_All_219716.h5ad')
adata

In [ ]:
from matplotlib import cm, colors
sc.settings.set_figure_params(dpi=300, figsize = (4, 4), fontsize=12)

# 上皮
colors2 = plt.cm.Greens(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['EPCAM'], frameon=False, color_map=mymap)


# T
colors2 = plt.cm.Blues(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['CD3D'], frameon=False, color_map=mymap)


# B
colors2 = plt.cm.Purples(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['CD79A'], frameon=False, color_map=mymap)


# MNPs
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['IGSF6'], frameon=False, color_map=mymap)


# Mast
colors2 = plt.cm.Oranges(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['TPSAB1'], frameon=False, color_map=mymap)


# Stromal
colors2 = plt.cm.Greys(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
sc.pl.umap(adata, color=['IGFBP7'], frameon=False, color_map=mymap)

In [ ]:
adata

In [ ]:
ax = sc.pl.correlation_matrix(adata, 'annotation', figsize=(15,15))